# MSc in Data Analytics
# Sentiment Analysis

Assessment Task
Students are advised to review and adhere to the submission requirements documented after the assessment task.  
In this continuous assessment, You are required to identify and carry out an analysis of a large dataset gleaned from the twitter API. Instructions for accessing the data can be found here  
https://datascienceparichay.com/article/get-data-from-twitter-api-in-python-step-by-step-guide/  
https://www.toptal.com/apache/apache-spark-streaming-twitter  
OR You may use the data held here:  
https://archive.org/details/twitterstream?sort=-publicdate  
You must collect at least 1 year's tweets on a topic, this data should be stored as requested below, and you are then required to analyse any change sentiment that occurs over the time period that you have selected.  
Following your analysis, you are then required to make a time series forecast of the sentiment at 1 week, 1 month and 3 months going forward. This forecast must be displayed as a dynamic dashboard.   


In [1]:
import findspark
findspark.init()

In [2]:
from pyspark import SparkContext, SparkConf
from pyspark.sql import SQLContext
from pyspark.sql.functions import col
import json, os

In [3]:
# Parameters

# Topic
topic = "vaccine"

# Read the directory and find the bz2 files containint the tweets
dataset_path = r'/home/mara/datasets/twitter'


# Size of the batch 
num_files = 2

destination_folder = f'/home/mara/datasets/topic'

In [4]:
# Create the Spark Context
# fix to manage the error 
# "Truncated the string representation of a plan since it was too large. This behavior can be adjusted by setting 'spark.debug.maxToStringFields' in SparkEnv.conf.

from pyspark.sql import SparkSession

spark = SparkSession.builder.appName("sba22243-step1").config("spark.sql.debug.maxToStringFields", 100).getOrCreate()
sqlContext = spark._wrapped

2023-05-11 23:34:20,720 WARN util.Utils: Your hostname, PAVILION resolves to a loopback address: 127.0.1.1; using 172.20.224.106 instead (on interface eth0)
2023-05-11 23:34:20,721 WARN util.Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
2023-05-11 23:34:22,228 WARN util.NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [8]:
dataframe = sqlContext.read.json('file:///home/mara/datasets/topic/vaccine-January-2021.json')

In [9]:
dataframe.show(5)

+-----------+--------------------+------------------+--------------------+--------------+---------+------------+----+-------------------+-------------------+-----------------------+---------------------+-------------------------+-------------------+-----------------------+---------------+----+-----+------------------+-----------+-------------+----------------+--------------------+-----------------------+-----------+-------------+---------+--------------------+--------------------+-------------+---------+--------------------+---------------------+
|coordinates|          created_at|display_text_range|      extended_tweet|favorite_count|favorited|filter_level| geo|                 id|             id_str|in_reply_to_screen_name|in_reply_to_status_id|in_reply_to_status_id_str|in_reply_to_user_id|in_reply_to_user_id_str|is_quote_status|lang|place|possibly_sensitive|quote_count|quoted_status|quoted_status_id|quoted_status_id_str|quoted_status_permalink|reply_count|retweet_count|retweeted|    

In [11]:
dataframe.printSchema()

root
 |-- coordinates: struct (nullable = true)
 |    |-- coordinates: array (nullable = true)
 |    |    |-- element: double (containsNull = true)
 |    |-- type: string (nullable = true)
 |-- created_at: string (nullable = true)
 |-- display_text_range: array (nullable = true)
 |    |-- element: long (containsNull = true)
 |-- extended_tweet: struct (nullable = true)
 |    |-- display_text_range: array (nullable = true)
 |    |    |-- element: long (containsNull = true)
 |    |-- entities: struct (nullable = true)
 |    |    |-- hashtags: array (nullable = true)
 |    |    |    |-- element: struct (containsNull = true)
 |    |    |    |    |-- indices: array (nullable = true)
 |    |    |    |    |    |-- element: long (containsNull = true)
 |    |    |    |    |-- text: string (nullable = true)
 |    |    |-- media: array (nullable = true)
 |    |    |    |-- element: struct (containsNull = true)
 |    |    |    |    |-- additional_media_info: struct (nullable = true)
 |    |    |  